In [28]:
import psycopg2

DB_CONFIG = {
    'dbname': 'demo',
    'user': 'postgres', 
    'password': '01072004',
    'host': 'localhost',
    'port': '5432'
}

def zadanie_1():
    """Задание 1: Структура таблиц БД"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Получаем все таблицы
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'bookings'
        ORDER BY table_name
    """)
    
    tables = [table[0] for table in cur.fetchall()]
    
    # Для каждой таблицы получаем структуру
    for table in tables:
        print(f"Таблица: {table}")
        
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'bookings' AND table_name = '{table}'
            ORDER BY ordinal_position
        """)
        
        columns = cur.fetchall()
        for col in columns:
            print(f"  {col[0]} ({col[1]})")
        print()
    
    cur.close()
    conn.close()

# Описание таблиц
table_descriptions = """
ОПИСАНИЕ ТАБЛИЦ:

airplanes - Данные о самолетах (коды, модели, характеристики)
airplanes_data - Технические данные самолетов
airports_data - Информация об аэропортах
boarding_passes - Посадочные талоны
bookings - Бронирования
flights - Рейсы
routes - Маршруты
seats - Места в самолетах
segments - Сегменты перелетов
tickets - Билеты
"""

print("ЗАДАНИЕ 1: Структура таблиц БД")
print("=" * 50)
zadanie_1()
print(table_descriptions)

ЗАДАНИЕ 1: Структура таблиц БД
Таблица: airplanes
  airplane_code (character)
  model (text)
  range (integer)
  speed (integer)

Таблица: airplanes_data
  airplane_code (character)
  model (jsonb)
  range (integer)
  speed (integer)

Таблица: airports
  airport_code (character)
  airport_name (text)
  city (text)
  country (text)
  coordinates (point)
  timezone (text)

Таблица: airports_data
  airport_code (character)
  airport_name (jsonb)
  city (jsonb)
  country (jsonb)
  coordinates (point)
  timezone (text)

Таблица: boarding_passes
  ticket_no (text)
  flight_id (integer)
  seat_no (text)
  boarding_no (integer)
  boarding_time (timestamp with time zone)

Таблица: bookings
  book_ref (character)
  book_date (timestamp with time zone)
  total_amount (numeric)

Таблица: flights
  flight_id (integer)
  route_no (text)
  status (text)
  scheduled_departure (timestamp with time zone)
  scheduled_arrival (timestamp with time zone)
  actual_departure (timestamp with time zone)
  actua

In [29]:
def zadanie_2():
    """Задание 2: Информация о таблицах и количестве записей"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'bookings'
        ORDER BY table_name
    """)
    
    tables = [table[0] for table in cur.fetchall()]
    table_counts = {}
    
    for table in tables:
        cur.execute(f'SELECT COUNT(*) FROM bookings."{table}"')
        count = cur.fetchone()[0]
        table_counts[table] = count
    
    # Таблица с максимальным количеством записей
    max_table = max(table_counts, key=table_counts.get)
    
    print("Количество записей в таблицах:")
    for table, count in table_counts.items():
        print(f"  {table}: {count}")
    
    print(f"Таблица с максимальным количеством записей: {max_table} ({table_counts[max_table]})")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 2: Информация о таблицах")
print("=" * 50)
zadanie_2()


ЗАДАНИЕ 2: Информация о таблицах
Количество записей в таблицах:
  airplanes: 20
  airplanes_data: 20
  airports: 11002
  airports_data: 11002
  boarding_passes: 4927664
  bookings: 2585786
  flights: 43516
  routes: 2324
  seats: 3482
  segments: 3941249
  tickets: 2973937
  timetable: 348128
Таблица с максимальным количеством записей: boarding_passes (4927664)


In [30]:
def zadanie_3():
    """Задание 3: Названия тарифов"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("SELECT DISTINCT fare_conditions FROM bookings.seats ORDER BY fare_conditions")
    fares = cur.fetchall()
    
    print("Названия тарифов:")
    for fare in fares:
        print(f"  {fare[0]}")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 3: Названия тарифов")
print("=" * 50)
zadanie_3()


ЗАДАНИЕ 3: Названия тарифов
Названия тарифов:
  Business
  Comfort
  Economy


In [31]:
def zadanie_4():
    """Задание 4: Выручка по тарифам"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT 
            fare_conditions,
            SUM(price) as total_revenue
        FROM bookings.segments 
        GROUP BY fare_conditions
        ORDER BY total_revenue DESC
    """)
    
    results = cur.fetchall()
    
    print("Выручка по тарифам:")
    for row in results:
        print(f"  {row[0]}: {row[1]:.2f} руб.")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 4: Выручка по тарифам")
print("=" * 50)
zadanie_4()


ЗАДАНИЕ 4: Выручка по тарифам
Выручка по тарифам:
  Economy: 17696017750.00 руб.
  Business: 4939418500.00 руб.
  Comfort: 1938495325.00 руб.


In [32]:
def zadanie_5():
    """Задание 5: Тариф с максимальным доходом"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT 
            fare_conditions,
            SUM(price) as total_revenue
        FROM bookings.segments 
        GROUP BY fare_conditions
        ORDER BY total_revenue DESC
        LIMIT 1
    """)
    
    result = cur.fetchone()
    
    print("Тариф с максимальным доходом:")
    print(f"  {result[0]}: {result[1]:.2f} руб.")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 5: Тариф с максимальным доходом")
print("=" * 50)
zadanie_5()


ЗАДАНИЕ 5: Тариф с максимальным доходом
Тариф с максимальным доходом:
  Economy: 17696017750.00 руб.


In [33]:
import time

def zadanie_5_alternative():
    """Задание 5: Поиск самолета с минимальной дальностью 2 способами"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Способ 1
    start_time = time.time()
    cur.execute("""
        SELECT airplane_code, model, range
        FROM bookings.airplanes
        WHERE range = (SELECT MIN(range) FROM bookings.airplanes)
    """)
    result1 = cur.fetchall()
    time1 = time.time() - start_time
    
    # Способ 2
    start_time = time.time()
    cur.execute("""
        SELECT airplane_code, model, range
        FROM bookings.airplanes
        ORDER BY range ASC
        LIMIT 1
    """)
    result2 = cur.fetchall()
    time2 = time.time() - start_time
    
    print("Самолет с минимальной дальностью:")
    print(f"  {result2[0][0]} - {result2[0][1]} - {result2[0][2]} км")
    print(f"Время выполнения способа 1: {time1:.4f} сек.")
    print(f"Время выполнения способа 2: {time2:.4f} сек.")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 5: Поиск самолета с минимальной дальностью")
print("=" * 50)
zadanie_5_alternative()


ЗАДАНИЕ 5: Поиск самолета с минимальной дальностью
Самолет с минимальной дальностью:
  CR7 - Bombardier CRJ700 - 3100 км
Время выполнения способа 1: 0.0023 сек.
Время выполнения способа 2: 0.0004 сек.


In [34]:
def zadanie_6():
    """Задание 6: Рейсы с максимальной длительностью"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT MAX(scheduled_arrival - scheduled_departure) as max_duration
        FROM bookings.flights
    """)
    max_duration = cur.fetchone()[0]
    
    cur.execute("""
        SELECT COUNT(*) as flight_count
        FROM bookings.flights
        WHERE (scheduled_arrival - scheduled_departure) = %s
    """, (max_duration,))
    flight_count = cur.fetchone()[0]
    
    print(f"Максимальная длительность полета: {max_duration}")
    print(f"Количество рейсов с максимальной длительностью: {flight_count}")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 6: Рейсы с максимальной длительностью")
print("=" * 50)
zadanie_6()


ЗАДАНИЕ 6: Рейсы с максимальной длительностью
Максимальная длительность полета: 19:55:00
Количество рейсов с максимальной длительностью: 18


In [35]:
def zadanie_7():
    """Задание 7: Маршруты с максимальной длительностью"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT MAX(scheduled_arrival - scheduled_departure) as max_duration
        FROM bookings.flights
    """)
    max_duration = cur.fetchone()[0]
    
    cur.execute("""
        SELECT DISTINCT 
            r.departure_airport,
            dep_air.airport_name->>'ru' as departure_airport_name,
            dep_air.city->>'ru' as departure_city,
            r.arrival_airport,
            arr_air.airport_name->>'ru' as arrival_airport_name,
            arr_air.city->>'ru' as arrival_city
        FROM bookings.flights f
        JOIN bookings.routes r ON f.route_no = r.route_no
        JOIN bookings.airports_data dep_air ON r.departure_airport = dep_air.airport_code
        JOIN bookings.airports_data arr_air ON r.arrival_airport = arr_air.airport_code
        WHERE (f.scheduled_arrival - f.scheduled_departure) = %s
    """, (max_duration,))
    
    routes = cur.fetchall()
    
    print("Маршруты с максимальной длительностью:")
    for route in routes:
        print(f"  {route[1]} ({route[2]}) -> {route[4]} ({route[5]}): {max_duration}")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 7: Маршруты с максимальной длительностью")
print("=" * 50)
zadanie_7()


ЗАДАНИЕ 7: Маршруты с максимальной длительностью
Маршруты с максимальной длительностью:
  Артуро Мерино Бенитес (Сантьяго) -> Чхатрапати Шиваджи (Мумбаи): 19:55:00
  Чхатрапати Шиваджи (Мумбаи) -> Артуро Мерино Бенитес (Сантьяго): 19:55:00


In [36]:
def zadanie_8():
    """Задание 8: Аэропорт с максимальной нагрузкой"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        WITH airport_operations AS (
            SELECT r.departure_airport as airport_code, COUNT(*) as operations
            FROM bookings.flights f JOIN bookings.routes r ON f.route_no = r.route_no
            GROUP BY r.departure_airport
            UNION ALL
            SELECT r.arrival_airport as airport_code, COUNT(*) as operations
            FROM bookings.flights f JOIN bookings.routes r ON f.route_no = r.route_no
            GROUP BY r.arrival_airport
        )
        SELECT 
            ao.airport_code,
            a.airport_name->>'ru' as airport_name,
            a.city->>'ru' as city,
            SUM(ao.operations) as total_operations
        FROM airport_operations ao
        JOIN bookings.airports_data a ON ao.airport_code = a.airport_code
        GROUP BY ao.airport_code, a.airport_name, a.city
        ORDER BY total_operations DESC
        LIMIT 1
    """)
    
    result = cur.fetchone()
    
    print("Аэропорт с максимальной нагрузкой:")
    print(f"  {result[1]} ({result[2]}), {result[3]} операций")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 8: Аэропорт с максимальной нагрузкой")
print("=" * 50)
zadanie_8()


ЗАДАНИЕ 8: Аэропорт с максимальной нагрузкой
Аэропорт с максимальной нагрузкой:
  Франкфурт-на-Майне (Франкфурт-на-Майне), 33680 операций


In [37]:
def zadanie_9():
    """Задание 9: Среднее количество мест по классам"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT 
            fare_conditions,
            ROUND(AVG(seat_count), 2) as avg_seat_count
        FROM (
            SELECT airplane_code, fare_conditions, COUNT(*) as seat_count
            FROM bookings.seats
            GROUP BY airplane_code, fare_conditions
        ) as seat_counts
        GROUP BY fare_conditions
        ORDER BY avg_seat_count DESC
    """)
    
    results = cur.fetchall()
    
    print("Среднее количество мест по классам:")
    for row in results:
        print(f"  {row[0]}: {row[1]:.2f}")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 9: Среднее количество мест по классам")
print("=" * 50)
zadanie_9()


ЗАДАНИЕ 9: Среднее количество мест по классам
Среднее количество мест по классам:
  Economy: 356.25
  Comfort: 54.50
  Business: 51.75


In [38]:
def zadanie_10():
    """Задание 10: Самый дорогой перелет"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("""
        WITH flight_revenue AS (
            SELECT f.flight_id, SUM(s.price) as final_amount
            FROM bookings.segments s
            JOIN bookings.flights f ON s.flight_id = f.flight_id
            WHERE s.price IS NOT NULL
            GROUP BY f.flight_id
            ORDER BY final_amount DESC
            LIMIT 1
        )
        SELECT 
            fr.flight_id,
            fr.final_amount,
            r.departure_airport,
            dep_air.airport_name->>'ru' as departure_airport_name,
            dep_air.city->>'ru' as departure_city,
            r.arrival_airport,
            arr_air.airport_name->>'ru' as arrival_airport_name,
            arr_air.city->>'ru' as arrival_city
        FROM flight_revenue fr
        JOIN bookings.flights f ON fr.flight_id = f.flight_id
        JOIN bookings.routes r ON f.route_no = r.route_no
        JOIN bookings.airports_data dep_air ON r.departure_airport = dep_air.airport_code
        JOIN bookings.airports_data arr_air ON r.arrival_airport = arr_air.airport_code
    """)
    
    result = cur.fetchone()
    
    print("Самый дорогой перелет:")
    print(f"  ID рейса: {result[0]}")
    print(f"  Выручка: {result[1]:.2f} руб.")
    print(f"  Маршрут: {result[3]} ({result[4]}) -> {result[6]} ({result[7]})")
    
    cur.close()
    conn.close()

print("\nЗАДАНИЕ 10: Самый дорогой перелет")
print("=" * 50)
zadanie_10()


ЗАДАНИЕ 10: Самый дорогой перелет
Самый дорогой перелет:
  ID рейса: 16679
  Выручка: 45064200.00 руб.
  Маршрут: Окленд (Окленд) -> Шарлотт Дуглас (Шарлотт)
